In [126]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass
import json

import pandas as pd
import matplotlib.pyplot as plt


project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)
print(project_path)

from database_storage.helper import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.kpi import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/sam11/Git/nestor


# Database

In [127]:
user = "neo4j"
password="GREYSTONE!!"

In [128]:
databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)
database = DatabaseNeo4J("bolt://127.0.0.1:7687", user, password, databaseSchema_dict)

yaml file open


# Query

In [121]:
kpiIssue = OperandIssue(
                        databaseInfo=database.schema,
                        result = "description_of_problem"
                       )
kpiIssue1 = OperandIssue(
                        databaseInfo=database.schema,
                        result = "description_of_solution"
                       )
kpiHuman = OperandTechnician(property = "name", 
                        operator = "-", 
                        value = "steve",
                        databaseInfo=database.schema,
                        result = "name",
                       )
kpimachine = OperandMachine(property = "name", 
                        operator = "=", 
                        value = "h1",
                        databaseInfo=database.schema,
                       )
kpimachinetype = OperandMachine(property = "type", 
                        operator = "=", 
                        value = "h",
                        databaseInfo=database.schema,
                       )

kpiitem = OperandTagItem(result = "keyword",
                         databaseInfo=database.schema,
                       )

In [122]:
e = kpimachine + kpiitem + kpiHuman + kpiIssue + kpiIssue1

In [123]:
resultToObservationDataframe(database.runQuery(e.cypher_filterQuery())[1])

28it [00:00, 58225.34it/s]


,issue.description_of_solution,technician.name,tagitem.keyword,issue.description_of_problem
0,At 27bar -Charged to 30bar / Also replaced val...,steve andreozzi,accumulator,Accumulator check requested
1,At 27bar -Charged to 30bar / Also replaced val...,steve andreozzi,valve,Accumulator check requested
2,Repaired -Air tool oil added,steve andreozzi,tool oil,Loader not functioning properly (??) no details
3,Repaired -Air tool oil added,steve andreozzi,air tool,Loader not functioning properly (??) no details
4,Repaired -Air tool oil added,steve andreozzi,oil,Loader not functioning properly (??) no details
5,Repaired -Air tool oil added,steve andreozzi,tool,Loader not functioning properly (??) no details
6,Repaired -Air tool oil added,steve andreozzi,air,Loader not functioning properly (??) no details
7,Repaired -Air tool oil added,steve andreozzi,loader,Loader not functioning properly (??) no details
8,Replaced sensor on escapement,steve andreozzi,wire loader,Loose wire on loader
9,Replaced sensor on escapement,steve andreozzi,wire,Loose wire on loader


In [155]:
result = database.getAllPropertiesOf(":HUMAN")
result

['name']

In [131]:
print(e.cypher_filterQuery())

MATCH (issue:ISSUE)-[:COVERED]->(machine:MACHINE),(issue:ISSUE)-[:CONTAINS]->(tagitem:TAG:ONE_GRAM:ITEM),(issue:ISSUE)-[:SOLVE_BY]->(technician:HUMAN:TECHNICIAN),(issue:ISSUE)
WHERE  machine.name = "h1" AND  technician.name CONTAINS "steve"
RETURN issue.description_of_solution,technician.name,tagitem.keyword,issue.description_of_problem


In [110]:
print(kpiIssue)
print(kpiHuman)

MATCH issue.description_of_problem 1 "the valve is brkoen" RETURN description_of_problem
MATCH operator.name - "steve" RETURN name


In [18]:
for r in kpiIssue.cypher_filter():
    print(r)

(issue:ISSUE)
issue.description_of_problem IS NOT NULL
issue.description_of_problem


-----------